<br />
<p align="center">
  <img src="../images/dtlogo.png" alt="Logo" width="111" height="100">

  <h1 align="center">Object detection for robots</h1>
</p>


## Integration

Finally, you need to integrate your model with ROS. You will edit the object detection node found in `src/object_detection/src`.

If you don't have a Jetson Nano Duckiebot, you can run this exercise locally. This will run your code as a pytorch model

If you have a Jetson Nano Duckiebot, you can run this exercise on your Duckiebot. This will convert your model to a TensorRT model, and run it your Jetson Nano's GPU.

In both cases, you need to edit the ROS node to decide how you will use the detections. Should you call your model on every image from your camera? Only once every 4-5 frames, to preserve your CPU? You might also want to change your robot's behaviour depending on the size of the detection. If it is small, the object is probably far away, so there's no need to stop.

TODO: what should they do to get started? We will walk through each function that they should modify step by step


### Framerate

While object detection is useful, it is also very expensive, computationally.

One trick used to reduce said cost is to only call the full model infrequently.
For example, one might call the model only every second, which is very slow in
computer timeframes, but relatively fast for the real world. Odds are that the detections
obtained every second will look very similar.

Of course, this varies from application to application. In very dynamic, fast
robotic environments, clearly the model should be called more frequently.

You can fine-tune this yourself: the function bellow indicates the number of frames
your robot should skip before calling its object detection model.

In [ ]:
def NUMBER_FRAMES_SKIPPED():
    # todo change this number to drop more frames
    # (must be a positive integer)
    return 0

In real life, we would use a full neural network model to produce very accurate
predictions, and then a less accurate model coupled with a Kalman filter (or
other such estimation system) to "track" each prediction during the skipped frames.

For this exercises, we will limit ourselves to just skipping frames.

### Filtering

Some of your predictions might not actually be useful. For example, in Duckietown,
the trucks and busses are always parked on the side of the road. Your robot will
never have to avoid or stop for them.

Cones can be in the road in some maps, but for this exercises, you can assume that there
aren't any.

#### Filtering by class

For this reason, you probably want to remove all non-duckies from your predictions,
since only duckies will be on the road.

In [ ]:
# `class` is the class of a prediction
def filter_by_classes(clas):
    # Right now, this returns True for every object's class
    # Change this to only return True for duckies!
    # In other words, returning False means that this prediction is ignored.
    return True

#### Filtering by score

Depending on the model, you might also want to remove very unconfident detections.

Then again, your model might not be confident even for detections that are absolutely
correct. You should experiment to find a value that works well for your model.

In [ ]:
# `scor` is the confidence score of a prediction
def filter_by_scores(scor):
    # Right now, this returns True for every object's confidence
    # Change this to filter the scores, or not at all
    # (returning True for all of them might be the right thing to do!)
    return True

#### Filtering by bounding box

Finally, you should also evaluate what each detection *means* in terms of positionning.

If a bounding box is in the leftmost or rightmost thirds of the image, it might be the case that
the object in not even on the road, and that your robot would be able to go by it without issue.

![image of a bounding box](../images/thirds.png)

Also, if a bounding box's area is small, its object is likely far away. So there is no need to
try to avoid the object or stop the robot: the robot still has a bit of driving to do before it reaches
the object. So filtering out small detections might be a good idea too.

In [ ]:
# `bbox` is the bounding box of a prediction, in xyxy format
# So it is of the shape (leftmost x pixel, topmost y pixel, rightmost x pixel, bottommost y pixel)
def filter_by_bboxes(bbox):
    # Like in the other cases, return False if the bbox should not be considered.
    return True


### Fine-tuning

In all of the functions above, there is not objective right answer. You should play with
your functions and fine-tune their behaviours. Robotics is an iterative process!